# LangGraph Agent Analytics with BigQuery

This notebook demonstrates how **BigQuery serves as a unified platform** for LangGraph Agent:
- **Observability** - Real-time event monitoring and tracing
- **Analytics** - Tool usage, latency analysis, and cost tracking
- **Evaluation** - Quality assessment and safety monitoring
- **Memory** - Semantic search across agent conversations

## Architecture

```
┌─────────────────┐     ┌──────────────────────────┐     ┌─────────────────┐
│  LangGraph      │────▶│  BigQueryCallbackHandler │────▶│    BigQuery     │
│  Agent          │     │  (Async Logging)         │     │  agent_events   │
└─────────────────┘     └──────────────────────────┘     └─────────────────┘
                                                                  │
                                                                  ▼
                                                         ┌─────────────────┐
                                                         │  Analytics &    │
                                                         │  Dashboards     │
                                                         └─────────────────┘
```

## Setup & Authentication

In [ ]:
# Install required packages (uncomment if needed)
# !pip install google-cloud-bigquery pandas matplotlib seaborn

In [ ]:
# Authentication for Google Colab
try:
    from google.colab import auth
    auth.authenticate_user()
    print("Authenticated with Google Colab")
except ImportError:
    print("Not running in Colab - using application default credentials")

In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuration - UPDATE THESE FOR YOUR ENVIRONMENT
PROJECT_ID = "test-project-0728-467323"  # Your GCP project
DATASET_ID = "agent_analytics"           # Your BigQuery dataset
TABLE_ID = "agent_events_v2"             # Your events table

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)
print(f"Connected to BigQuery: {PROJECT_ID}.{DATASET_ID}.{TABLE_ID}")

In [ ]:
# Helper function to run queries
def run_query(sql: str) -> pd.DataFrame:
    """Execute a BigQuery SQL query and return results as a DataFrame."""
    return client.query(sql).to_dataframe()

# Set display options
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 50)
sns.set_theme(style="whitegrid")

## Table Schema

The `agent_events_v2` table captures all LangGraph agent events with the following schema:

| Column | Type | Description |
|--------|------|-------------|
| `timestamp` | TIMESTAMP | Event timestamp |
| `event_type` | STRING | LLM_REQUEST, LLM_RESPONSE, TOOL_STARTING, TOOL_COMPLETED, NODE_STARTING, GRAPH_START, GRAPH_END, etc. |
| `agent` | STRING | Agent name (e.g., finance_assistant, travel_planner) |
| `session_id` | STRING | Session identifier for conversation tracking |
| `user_id` | STRING | User identifier |
| `trace_id` | STRING | Distributed trace ID |
| `span_id` | STRING | Span ID within trace |
| `content` | JSON | Event content (prompts, responses, tool args/results) |
| `attributes` | JSON | Additional attributes (tool_name, langgraph metadata) |
| `latency_ms` | JSON | Latency measurements |
| `status` | STRING | OK, ERROR |

---
# Phase 1: Real-time Observability

Monitor agent activity in real-time, track events, and understand the flow of requests.

### 1.1 Recent Event Stream

View the most recent agent events across all agents.

In [ ]:
recent_events_sql = f"""
SELECT 
    timestamp,
    agent,
    event_type,
    session_id,
    user_id,
    JSON_EXTRACT_SCALAR(attributes, '$.tool_name') as tool_name,
    CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS INT64) as latency_ms,
    status
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
ORDER BY timestamp DESC
LIMIT 30
"""

recent_events = run_query(recent_events_sql)
print(f"Recent {len(recent_events)} events:")
recent_events

### 1.2 Event Volume by Type

Understand the distribution of event types across your agents.

In [ ]:
event_distribution_sql = f"""
SELECT 
    event_type,
    COUNT(*) as count,
    COUNT(DISTINCT session_id) as unique_sessions
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
GROUP BY event_type
ORDER BY count DESC
"""

event_dist = run_query(event_distribution_sql)

# Visualization
if len(event_dist) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = sns.color_palette("husl", len(event_dist))
    bars = ax.barh(event_dist['event_type'], event_dist['count'], color=colors)
    ax.set_xlabel('Event Count')
    ax.set_title('Event Type Distribution')
    ax.bar_label(bars, padding=3)
    plt.tight_layout()
    plt.show()
else:
    print("No event distribution data found for today.")

event_dist

### 1.3 Agent Activity Summary

Compare activity levels across different agents.

In [ ]:
agent_activity_sql = f"""
SELECT 
    agent,
    COUNT(*) as total_events,
    COUNT(DISTINCT session_id) as sessions,
    COUNT(DISTINCT user_id) as unique_users,
    COUNTIF(event_type = 'LLM_REQUEST') as llm_calls,
    COUNTIF(event_type = 'TOOL_STARTING') as tool_calls,
    COUNTIF(status = 'ERROR') as errors
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
  AND agent IS NOT NULL
GROUP BY agent
ORDER BY total_events DESC
"""

agent_activity = run_query(agent_activity_sql)
agent_activity

---
# Phase 2: Performance Analytics

Analyze latency, tool usage patterns, and identify performance bottlenecks.

### 2.1 Tool Usage Analysis

Which tools are being used most frequently and by which agents?

In [ ]:
tool_usage_sql = f"""
SELECT 
    agent,
    JSON_EXTRACT_SCALAR(attributes, '$.tool_name') as tool_name,
    COUNT(*) as call_count,
    AVG(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64)) as avg_latency_ms,
    MAX(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64)) as max_latency_ms,
    COUNTIF(status = 'ERROR') as error_count
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE event_type = 'TOOL_COMPLETED'
  AND DATE(timestamp) = CURRENT_DATE()
  AND JSON_EXTRACT_SCALAR(attributes, '$.tool_name') IS NOT NULL
GROUP BY agent, tool_name
ORDER BY call_count DESC
"""

tool_usage = run_query(tool_usage_sql)

# Visualization - Tool usage heatmap
if len(tool_usage) > 0 and tool_usage['agent'].notna().any() and tool_usage['tool_name'].notna().any():
    pivot_data = tool_usage.pivot_table(
        index='tool_name', 
        columns='agent', 
        values='call_count', 
        fill_value=0
    )
    
    # Only create heatmap if pivot_data has data
    if pivot_data.size > 0 and not pivot_data.empty:
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.heatmap(pivot_data, annot=True, fmt='g', cmap='YlOrRd', ax=ax)
        ax.set_title('Tool Usage by Agent')
        plt.tight_layout()
        plt.show()
    else:
        print("No tool usage data available for heatmap visualization.")
else:
    print("No tool usage data found for today.")

tool_usage

### 2.2 Latency Analysis

Understand latency distribution across LLM calls and tool executions.

In [ ]:
latency_analysis_sql = f"""
SELECT 
    event_type,
    agent,
    COUNT(*) as count,
    ROUND(AVG(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64)), 2) as avg_latency_ms,
    ROUND(APPROX_QUANTILES(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64), 100)[OFFSET(50)], 2) as p50_latency_ms,
    ROUND(APPROX_QUANTILES(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64), 100)[OFFSET(95)], 2) as p95_latency_ms,
    MAX(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS INT64)) as max_latency_ms
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
  AND event_type IN ('LLM_RESPONSE', 'TOOL_COMPLETED', 'GRAPH_END')
  AND JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') IS NOT NULL
GROUP BY event_type, agent
ORDER BY avg_latency_ms DESC
"""

latency_analysis = run_query(latency_analysis_sql)
latency_analysis

### 2.3 Graph Execution Analysis

Analyze complete graph executions (session-level metrics).

In [ ]:
graph_execution_sql = f"""
WITH graph_sessions AS (
    SELECT 
        session_id,
        agent,
        user_id,
        MIN(timestamp) as start_time,
        MAX(timestamp) as end_time,
        COUNTIF(event_type = 'LLM_REQUEST') as llm_calls,
        COUNTIF(event_type = 'TOOL_STARTING') as tool_calls,
        MAX(CASE WHEN event_type = 'GRAPH_END' 
            THEN CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS INT64) END) as total_latency_ms,
        COUNTIF(status = 'ERROR') as errors
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE DATE(timestamp) = CURRENT_DATE()
    GROUP BY session_id, agent, user_id
)
SELECT 
    session_id,
    agent,
    user_id,
    start_time,
    llm_calls,
    tool_calls,
    total_latency_ms,
    errors,
    CASE WHEN errors > 0 THEN 'Failed' ELSE 'Success' END as status
FROM graph_sessions
ORDER BY start_time DESC
LIMIT 20
"""

graph_execution = run_query(graph_execution_sql)
graph_execution

### 2.4 Cost Estimation

Estimate costs based on token usage (using approximate token counts from content length).

In [ ]:
cost_estimation_sql = f"""
WITH token_estimates AS (
    SELECT 
        agent,
        event_type,
        -- Rough estimate: 1 token ≈ 4 characters
        CEIL(LENGTH(TO_JSON_STRING(content)) / 4) as estimated_tokens
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE DATE(timestamp) = CURRENT_DATE()
      AND event_type IN ('LLM_REQUEST', 'LLM_RESPONSE')
)
SELECT 
    agent,
    SUM(CASE WHEN event_type = 'LLM_REQUEST' THEN estimated_tokens ELSE 0 END) as input_tokens,
    SUM(CASE WHEN event_type = 'LLM_RESPONSE' THEN estimated_tokens ELSE 0 END) as output_tokens,
    SUM(estimated_tokens) as total_tokens,
    -- Gemini 1.5 Flash pricing (approximate): $0.075/1M input, $0.30/1M output
    ROUND(SUM(CASE WHEN event_type = 'LLM_REQUEST' THEN estimated_tokens ELSE 0 END) * 0.000000075 +
          SUM(CASE WHEN event_type = 'LLM_RESPONSE' THEN estimated_tokens ELSE 0 END) * 0.0000003, 4) as estimated_cost_usd
FROM token_estimates
GROUP BY agent
ORDER BY total_tokens DESC
"""

cost_estimation = run_query(cost_estimation_sql)
cost_estimation

---
# Phase 3: Error Analysis & Debugging

Identify and analyze errors to improve agent reliability.

### 3.1 Error Summary

Overview of errors by type and agent.

In [ ]:
error_summary_sql = f"""
SELECT 
    agent,
    event_type,
    COUNT(*) as error_count,
    MIN(timestamp) as first_occurrence,
    MAX(timestamp) as last_occurrence
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE status = 'ERROR'
  AND DATE(timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
GROUP BY agent, event_type
ORDER BY error_count DESC
"""

error_summary = run_query(error_summary_sql)
if len(error_summary) > 0:
    print(f"Found {len(error_summary)} error types:")
    display(error_summary)
else:
    print("No errors found in the last 7 days!")

### 3.2 Error Details

Detailed view of recent errors with context.

In [ ]:
error_details_sql = f"""
SELECT 
    timestamp,
    agent,
    session_id,
    event_type,
    JSON_EXTRACT_SCALAR(attributes, '$.tool_name') as tool_name,
    SUBSTR(TO_JSON_STRING(content), 1, 200) as content_preview
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE status = 'ERROR'
  AND DATE(timestamp) = CURRENT_DATE()
ORDER BY timestamp DESC
LIMIT 10
"""

error_details = run_query(error_details_sql)
if len(error_details) > 0:
    display(error_details)
else:
    print("No errors today!")

---
# Phase 4: Session & Conversation Analysis

Reconstruct and analyze complete agent conversations.

### 4.1 Session Timeline

View the complete event timeline for a specific session.

In [ ]:
# First, get a sample session ID
sample_session_sql = f"""
SELECT DISTINCT session_id
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
  AND session_id IS NOT NULL
ORDER BY session_id DESC
LIMIT 1
"""

sample_session = run_query(sample_session_sql)
if len(sample_session) > 0:
    session_id = sample_session.iloc[0]['session_id']
    print(f"Analyzing session: {session_id}")
else:
    session_id = None
    print("No sessions found")

In [ ]:
if session_id:
    session_timeline_sql = f"""
    SELECT 
        timestamp,
        event_type,
        JSON_EXTRACT_SCALAR(attributes, '$.tool_name') as tool_name,
        CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS INT64) as latency_ms,
        status,
        SUBSTR(TO_JSON_STRING(content), 1, 200) as content_preview
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE session_id = '{session_id}'
    ORDER BY timestamp
    """
    
    session_timeline = run_query(session_timeline_sql)
    print(f"Session {session_id} - {len(session_timeline)} events:")
    display(session_timeline)

### 4.2 Conversation Reconstruction

Reconstruct the human-agent conversation from a session.

In [ ]:
if session_id:
    conversation_sql = f"""
    SELECT 
        timestamp,
        CASE 
            WHEN event_type = 'LLM_REQUEST' THEN '👤 User'
            WHEN event_type = 'LLM_RESPONSE' THEN '🤖 Agent'
            WHEN event_type = 'TOOL_STARTING' THEN '🔧 Tool Call'
            WHEN event_type = 'TOOL_COMPLETED' THEN '✅ Tool Result'
            ELSE event_type
        END as role,
        JSON_EXTRACT_SCALAR(attributes, '$.tool_name') as tool,
        SUBSTR(JSON_EXTRACT_SCALAR(content, '$.summary'), 1, 300) as message
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE session_id = '{session_id}'
      AND event_type IN ('LLM_REQUEST', 'LLM_RESPONSE', 'TOOL_STARTING', 'TOOL_COMPLETED')
    ORDER BY timestamp
    """
    
    conversation = run_query(conversation_sql)
    print(f"Conversation for session {session_id}:")
    for _, row in conversation.iterrows():
        tool_info = f" [{row['tool']}]" if pd.notna(row['tool']) else ""
        print(f"{row['role']}{tool_info}: {row['message'] or '(no content)'}")
        print()

---
# Phase 5: User Analytics

Understand user behavior and engagement patterns.

### 5.1 User Engagement Metrics

In [ ]:
user_engagement_sql = f"""
SELECT 
    user_id,
    COUNT(DISTINCT session_id) as total_sessions,
    COUNT(DISTINCT agent) as agents_used,
    COUNTIF(event_type = 'LLM_REQUEST') as total_queries,
    COUNTIF(event_type = 'TOOL_STARTING') as tool_interactions,
    ROUND(AVG(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64)), 0) as avg_response_time_ms,
    COUNTIF(status = 'ERROR') as errors_encountered
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
  AND user_id IS NOT NULL
GROUP BY user_id
ORDER BY total_sessions DESC
"""

user_engagement = run_query(user_engagement_sql)
user_engagement

### 5.2 Agent Preference by User

In [ ]:
user_agent_preference_sql = f"""
SELECT 
    user_id,
    agent,
    COUNT(DISTINCT session_id) as sessions,
    COUNT(*) as total_events
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
  AND user_id IS NOT NULL
  AND agent IS NOT NULL
GROUP BY user_id, agent
ORDER BY user_id, sessions DESC
"""

user_agent_pref = run_query(user_agent_preference_sql)

if len(user_agent_pref) > 0 and user_agent_pref['user_id'].notna().any() and user_agent_pref['agent'].notna().any():
    pivot = user_agent_pref.pivot_table(
        index='user_id', 
        columns='agent', 
        values='sessions', 
        fill_value=0
    )
    
    if pivot.size > 0 and not pivot.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        pivot.plot(kind='bar', ax=ax)
        ax.set_title('Agent Usage by User')
        ax.set_xlabel('User')
        ax.set_ylabel('Sessions')
        plt.legend(title='Agent')
        plt.tight_layout()
        plt.show()
    else:
        print("No user-agent preference data available for visualization.")
else:
    print("No user-agent preference data found for today.")

user_agent_pref

---
# Phase 6: Time-Series Analysis

Analyze trends over time.

### 6.1 Hourly Activity Pattern

In [ ]:
hourly_activity_sql = f"""
SELECT 
    EXTRACT(HOUR FROM timestamp) as hour,
    agent,
    COUNT(*) as event_count
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
  AND agent IS NOT NULL
GROUP BY hour, agent
ORDER BY hour, agent
"""

hourly_activity = run_query(hourly_activity_sql)

if len(hourly_activity) > 0 and hourly_activity['hour'].notna().any() and hourly_activity['agent'].notna().any():
    pivot = hourly_activity.pivot_table(
        index='hour', 
        columns='agent', 
        values='event_count', 
        fill_value=0
    )
    
    if pivot.size > 0 and not pivot.empty:
        fig, ax = plt.subplots(figsize=(12, 6))
        pivot.plot(kind='line', marker='o', ax=ax)
        ax.set_title('Hourly Activity by Agent')
        ax.set_xlabel('Hour of Day')
        ax.set_ylabel('Event Count')
        ax.set_xticks(range(24))
        plt.legend(title='Agent')
        plt.tight_layout()
        plt.show()
    else:
        print("No hourly activity data available for visualization.")
else:
    print("No hourly activity data found for today.")

### 6.2 Latency Trend

In [ ]:
latency_trend_sql = f"""
SELECT 
    TIMESTAMP_TRUNC(timestamp, HOUR) as hour,
    AVG(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64)) as avg_latency_ms,
    APPROX_QUANTILES(CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64), 100)[OFFSET(95)] as p95_latency_ms
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
  AND event_type = 'LLM_RESPONSE'
  AND JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') IS NOT NULL
GROUP BY hour
ORDER BY hour
"""

latency_trend = run_query(latency_trend_sql)

if len(latency_trend) > 0 and latency_trend['hour'].notna().any():
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(latency_trend['hour'], latency_trend['avg_latency_ms'], 'b-o', label='Avg Latency')
    ax.plot(latency_trend['hour'], latency_trend['p95_latency_ms'], 'r--o', label='P95 Latency')
    ax.set_title('LLM Response Latency Trend')
    ax.set_xlabel('Time')
    ax.set_ylabel('Latency (ms)')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No latency trend data found for today.")

---
# Summary Dashboard

Key metrics at a glance.

In [ ]:
summary_sql = f"""
SELECT 
    COUNT(*) as total_events,
    COUNT(DISTINCT session_id) as total_sessions,
    COUNT(DISTINCT user_id) as unique_users,
    COUNT(DISTINCT agent) as active_agents,
    COUNTIF(event_type = 'LLM_REQUEST') as llm_requests,
    COUNTIF(event_type = 'TOOL_STARTING') as tool_invocations,
    COUNTIF(status = 'ERROR') as total_errors,
    ROUND(COUNTIF(status = 'ERROR') * 100.0 / COUNT(*), 2) as error_rate_pct,
    ROUND(AVG(CASE WHEN event_type = 'LLM_RESPONSE' 
        THEN CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS FLOAT64) END), 0) as avg_llm_latency_ms
FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
WHERE DATE(timestamp) = CURRENT_DATE()
"""

summary = run_query(summary_sql)

print("=" * 60)
print("📊 LangGraph Agent Analytics - Daily Summary")
print("=" * 60)
print(f"\n📈 Volume Metrics:")
print(f"   Total Events:      {summary.iloc[0]['total_events']:,}")
print(f"   Total Sessions:    {summary.iloc[0]['total_sessions']:,}")
print(f"   Unique Users:      {summary.iloc[0]['unique_users']:,}")
print(f"   Active Agents:     {summary.iloc[0]['active_agents']}")
print(f"\n🤖 Agent Activity:")
print(f"   LLM Requests:      {summary.iloc[0]['llm_requests']:,}")
print(f"   Tool Invocations:  {summary.iloc[0]['tool_invocations']:,}")
print(f"\n⚡ Performance:")
print(f"   Avg LLM Latency:   {summary.iloc[0]['avg_llm_latency_ms']:,.0f} ms")
print(f"\n❌ Reliability:")
print(f"   Total Errors:      {summary.iloc[0]['total_errors']:,}")
print(f"   Error Rate:        {summary.iloc[0]['error_rate_pct']:.2f}%")
print("\n" + "=" * 60)

---
## Next Steps

1. **Set up scheduled queries** to run these analytics automatically
2. **Create Looker dashboards** for real-time monitoring
3. **Configure alerts** for error rate spikes or latency degradation
4. **Enable BQML** for advanced analytics (embeddings, sentiment analysis)

### Useful SQL Patterns

```sql
-- Get conversation context for a session
SELECT * FROM `project.dataset.agent_events_v2`
WHERE session_id = 'your-session-id'
ORDER BY timestamp;

-- Find slow LLM calls
SELECT * FROM `project.dataset.agent_events_v2`
WHERE event_type = 'LLM_RESPONSE'
  AND CAST(JSON_EXTRACT_SCALAR(latency_ms, '$.total_ms') AS INT64) > 5000;

-- Tool success rate
SELECT 
    JSON_EXTRACT_SCALAR(attributes, '$.tool_name') as tool,
    COUNTIF(status = 'OK') * 100.0 / COUNT(*) as success_rate
FROM `project.dataset.agent_events_v2`
WHERE event_type = 'TOOL_COMPLETED'
GROUP BY tool;
```